# Usabilidade

In [5]:
import pandas as pd
import numpy as np
import pickle
from sklearn.tree import DecisionTreeClassifier

# Data

In [2]:
df = pd.read_parquet('../data/encoded/fraud_dataset_v2_test.parquet.gzip')

In [7]:
model = pickle.load(open('../model/predictors/lgbm_model.pkl', 'rb'))

In [86]:
df['new_score'] = (model.predict_proba(df[model.feature_names_in_])[:,1]*100).round(0)
df['score'] = (df['score']).round(0)
df['monto_fraude'] = df['monto'] * df['fraude']
df['monto_not_fraude'] = df['monto'] * abs(df['fraude'] - 1)

In [87]:
df.head(2)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,fecha,monto,score,fraude,ymd,day_of_week,hour_of_day,period_of_day,vl_cat_last7d,qty_cat_last7d,vl_fraud_cat_last7d,qty_fraud_cat_last7d,br_vl_cat_last7d,br_qty_cat_last7d,new_score,monto_fraude,monto_not_fraude
0,4,0.7550,9258.5,1.0,0.000000,33.0,BR,0,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,0.376019,4008.0,0.0,1,0,N,0.02,9258,0,2020-04-15 19:58:08,124.71,72.0,0,20200415,2,19,evening,1070.50,63.0,46.75,2.0,0.043671,0.031746,7.0,0.0,124.71
1,4,0.7631,18923.9,50.0,0.482385,18.0,BR,23,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,0.154036,1704.0,1147.0,1,1,Y,0.98,18923,100,2020-04-16 16:03:10,3.54,2.0,0,20200416,3,16,afternoon,3.67,2.0,0.00,0.0,0.000000,0.000000,1.0,0.0,3.54


# Usabilidade

## Scores Separados

In [88]:
df_thresholds = (
    df
    .groupby(['new_score'])
    .agg({
        'monto' : ['count', 'sum'],
        'fraude' : 'sum',
        'monto_fraude' : 'sum',
        'monto_not_fraude' : 'sum'
    })
    .sort_values(by=['new_score'], ascending=False)
    .cumsum()
)
df_thresholds['monto_not_fraude_aprovada'] = df_thresholds['monto_not_fraude'].max() - df_thresholds['monto_not_fraude']
df_thresholds['monto_fraude_aprovada'] = df_thresholds['monto_fraude'].max() - df_thresholds['monto_fraude']
df_thresholds['roi'] = df_thresholds['monto_not_fraude_aprovada'] * 0.05 - df_thresholds['monto_fraude_aprovada']

df_thresholds.columns = ['qty_trans', 'vl_trans', 'qty_fraud', 'vl_fraud', 'vl_not_fraud', 'vl_not_fraud_approved', 'vl_fraud_approved', 'roi']

In [89]:
df_thresholds_old = (
    df
    .groupby(['score'])
    .agg({
        'monto' : ['count', 'sum'],
        'fraude' : 'sum',
        'monto_fraude' : 'sum',
        'monto_not_fraude' : 'sum'
    })
    .sort_values(by=['score'], ascending=False)
    .cumsum()
)
df_thresholds_old['monto_not_fraude_aprovada'] = df_thresholds_old['monto_not_fraude'].max() - df_thresholds_old['monto_not_fraude']
df_thresholds_old['monto_fraude_aprovada'] = df_thresholds_old['monto_fraude'].max() - df_thresholds_old['monto_fraude']
df_thresholds_old['roi'] = df_thresholds_old['monto_not_fraude_aprovada'] * 0.05 - df_thresholds_old['monto_fraude_aprovada']

df_thresholds_old.columns = ['qty_trans', 'vl_trans', 'qty_fraud', 'vl_fraud', 'vl_not_fraud', 'vl_not_fraud_approved', 'vl_fraud_approved', 'roi']

In [90]:
df_thresholds[df_thresholds['roi'] == df_thresholds['roi'].max()]

,qty_trans,vl_trans,qty_fraud,vl_fraud,vl_not_fraud,vl_not_fraud_approved,vl_fraud_approved,roi
new_score,,,,,,,,
4.0,18668,939276.79,1940,150389.66,788887.13,2679386.45,30280.64,103688.6825


In [91]:
df_thresholds_old[df_thresholds_old['roi'] == df_thresholds_old['roi'].max()]

,qty_trans,vl_trans,qty_fraud,vl_fraud,vl_not_fraud,vl_not_fraud_approved,vl_fraud_approved,roi
score,,,,,,,,
77.0,13156,722680.25,1456,113626.81,609053.44,2859220.14,67043.49,75917.517


Observando o ROI, chegamos a conclusão de que o novo score tem um ponto de corte que traz mais lucro para a empresa, aumentando em 27k, 36%.  Além de maior captura de fraude em quantidades, aumentando em 32%.  